This code accesses the Census dataset and creates CSV files with median house value, median household income, racial demographics, and educational attainment data for all census tracts in the New Jersey Mercer County area.

In [ ]:
!pip install census
!pip install us

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.9/356.9 kB 11.0 MB/s eta 0:00:00


In [ ]:
from census import Census
from us import states
import pandas as pd

import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dir = "/path/to/your/data" # Replace with actual file directory

In [ ]:
# Configure retries for the requests session
session = requests.Session()
retries = Retry(total=5, backoff_factor=0.1, status_forcelist=[500, 502, 503, 504])
session.mount('http://', HTTPAdapter(max_retries=retries))
session.mount('https://', HTTPAdapter(max_retries=retries))


In [ ]:
# Census API key
# request a key from https://api.census.gov/data/key_signup.html
CENSUS_API_KEY = "(Your Census Key)"
c = Census(CENSUS_API_KEY)

In [ ]:
# Federal Information Processing Standard codes (FIPS)
# Change to your state's or county's corresponding FIPS code
state_fips = states.NJ.fips  # New Jersey state FIPS (34)
county_fips = "021"          # Mercer County FIPS (021)

# Define valid years for Census API (2012 to 2022)
valid_years = range(2012, 2023)

# Functions

In [ ]:
from census import Census
from us import states
import pandas as pd

def fetch_census_data_by_field(state_fips, county_fips, valid_years, field, field_name):
    """
    Fetches data for a specific field (e.g., Median Household Income or Median House Value)
    for a specific state and county for each year in the valid_years range.

    Parameters:
    - state_fips: FIPS code for the state (e.g., "34" for New Jersey)
    - county_fips: FIPS code for the county (e.g., "021" for Mercer County)
    - valid_years: List or range of years to fetch data for (e.g., range(2012,2023) for 2012 to 2022)
    - field: The field code to query (e.g., 'B19013_001E')
    - field_name: The name of the field (e.g., 'Median_Household_Income')

    Returns:
    - data_all: DataFrame with data for the specified field for the specified years
    """
    # Initialize an empty DataFrame to store yearly data
    data_all = pd.DataFrame()

    # Fetch data for each year
    for year in valid_years:
        try:
            # Fetch data for the given field and year using acs5 (5-year estimates)
            data = c.acs5.state_county_tract(
                state_fips=state_fips,
                county_fips=county_fips,
                tract=Census.ALL,  # All Census Tracts
                year=year,  # Dynamically specify the year
                fields=(field,)  # The field to query (e.g., Median Household Income or Median House Value)
            )

            # Check if the data returned is empty or missing
            if not data:
                print(f"No data found for {field_name} in {year}")
                continue

            # Convert the data to a DataFrame
            df = pd.DataFrame(data)

            # Rename columns for clarity
            df.rename(columns={
                field: f'{field_name}_{year}',  # Add year to column name
                'state': 'State_FIPS',
                'county': 'County_FIPS',
                'tract': 'Census_Tract'  # Ensure tract is renamed correctly
            }, inplace=True)

            # Merge yearly data
            if data_all.empty:
                data_all = df
            else:
                data_all = pd.merge(data_all, df, on=['State_FIPS', 'County_FIPS', 'Census_Tract'], how='outer')

        except Exception as e:
            print(f"Error fetching data for {field_name} in {year}: {e}")

    return data_all


In [ ]:
def prepare_yearly_data(df, valid_years, field_name):
    """
    Prepares a dataset by:
    - Adding a Unique_ID column from State_FIPS, County_FIPS, and Census_Tract
    - Reordering columns with Unique_ID first
    - Keeping only the Unique_ID, location identifiers, and <field_name>_* columns for valid years

    Parameters
    - df: DataFrame containing the census data.
    - valid_years: List or range of years to fetch data for (e.g., range(2012,2023) for 2012 to 2022)
    - field_name: The name of the field (e.g., 'Median_Household_Income')

    Returns:
    - final_data: A cleaned and reordered DataFrame.
    """
    final_data = df.copy()

    # Generate Unique ID directly in the table
    final_data['Unique_ID'] = (
        final_data['State_FIPS'].astype(str) +
        final_data['County_FIPS'].astype(str) +
        final_data['Census_Tract'].astype(str)
    )

    # Reorder columns to place Unique_ID first
    cols = ['Unique_ID'] + [col for col in final_data.columns if col != 'Unique_ID']
    final_data = final_data[cols]

    # Define desired column order
    desired_column_order = ['Unique_ID', 'State_FIPS', 'County_FIPS', 'Census_Tract']

    # Dynamically add the specified field's columns for the valid years
    for year in valid_years:
        desired_column_order.append(f'{field_name}_{year}')

    # Keep only desired columns if they exist in the DataFrame
    final_data = final_data[[col for col in desired_column_order if col in final_data.columns]]

    return final_data


The U.S. Census Bureau revises geographic boundaries of census tracts every 10 years.

Certain census tracts are split into several census tracts in order to better reflect changes in population distribution, income, etc.

For example, tract 3302 was split into 3303 and 3304 in the year 2020.

As a result, tract 3302 only contains data from 2012-2019, while tracts 3303 and 3304 only contain data from 2020-2022.

In [ ]:
"""
This function is applied to the median household income and median
house value.

The new census tracts are updated with the old census tract values for
the period provided. The old census tract is then removed from the
dataset.

The reason for this method is because medians are positional statistics
rather than sums, like with populations, so estimating a ratio would not
reflect actual values.

The Census Bureau also does not offer granular data, so the median can
not be calculated for each census tract.


The new and old census tract IDs are manually obtained from the Census
Bureau.

As with the state_fips and county_fips, they should be changed to
reflect any changes in boundaries in the state or county.
"""



def update_median_census_tracts(final_data, tract_values, columns_to_update, years):
    """
    Update the values of median-based columns for specified census tracts in the
    DataFrame using the old census tract data.

    Parameters:
    - final_data: DataFrame containing the census data.
    - tract_values: Dictionary mapping new census tracts to old row values.
    - columns_to_update: List of column names to update.
    - years: The years for which the columns are updated. --- the first year cannot be before 2010, and the last year cannot be after 2019 (maximum would be range(2010, 2020))

    Returns:
    - Updated DataFrame with the specified values replaced.
    """

    # Update new census tracts with old census tract values
    for new_tracts, old_row in tract_values.items():
        for new_tract in new_tracts:  # Iterate over the tuple of new tracts
            for column in columns_to_update:
                # Determine the year of this column
                year = int(column.split('_')[-1])
                if year in years:
                    # Only copy old data for years before 2020
                    final_data.loc[final_data['Census_Tract'] == new_tract, column] = old_row[column]

    # Remove the old census tracts from the DataFrame
    old_tracts_to_remove = [old_row['Census_Tract'] for _, old_row in tract_values.items()]
    final_data = final_data[~final_data['Census_Tract'].isin(old_tracts_to_remove)]

    return final_data

In [ ]:
"""
This function is applied to the population statistics.

The ratio between the new census tracts for the year 2020 is calculated,
after which the old census tract values for the period provided are
proportionally distributed to update the new census tracts using this
ratio.

The old census tract is then removed from the dataset.

Since populations are sums, estimating the population distribution in
the new census tracts is feasible and able to more accurately reflect
actual values than if they were to be copied over from the old census
tracts or removed entirely.


The new and old census tract IDs are manually obtained from the Census
Bureau.

As with the state_fips and county_fips, they should be changed to
reflect any changes in boundaries in the state or county.
"""



def update_sum_census_tracts(final_data, tract_values, columns_to_update, years):
    """
    Update the values for sum-related columns for specified census tracts in the
    DataFrame using their ratios from the year 2020 and the old census tract data.

    Parameters:
    - final_data: DataFrame containing the census data.
    - tract_values: Dictionary mapping new census tracts to old row values.
    - columns_to_update: List of column names to update.
    - years: The years for which the columns are updated. --- the first year cannot be before 2010, and the last year cannot be after 2019 (maximum would be range(2010, 2020))

    Returns:
    - Updated DataFrame with the specified values replaced.
    """
    # Iterate through new tracts and proportionally distribute old data
    for new_tracts, old_row in tract_values.items():
        new_rows = [final_data[final_data['Census_Tract'] == tract].iloc[0] for tract in new_tracts]
        for column in columns_to_update:
            # Determine the year of this column
            year = int(column.split('_')[-1])
            if year in years:
                total_2020 = sum(row[column[:-4] + '2020'] for row in new_rows)

                # Avoid division by zero
                ratios = [row[column[:-4] + '2020'] / total_2020 if total_2020 else 1/len(new_rows) for row in new_rows] # if total_2020 = 0 (meaning that the new census tracts had a sum population of 0), use an even split ratio among all tracts

                for new_tract, ratio in zip(new_tracts, ratios):
                    final_data.loc[final_data['Census_Tract'] == new_tract, column] = old_row[column] * ratio


    # Remove the old census tracts from the DataFrame
    old_tracts_to_remove = [old_row['Census_Tract'] for _, old_row in tract_values.items()]
    final_data = final_data[~final_data['Census_Tract'].isin(old_tracts_to_remove)]

    return final_data

In [ ]:
def calculate_percentages(df, valid_years, categories, total_prefix):
    """
    Calculates percentage values for multiple categories based on a total value for each year,
    applied to each row in a Pandas DataFrame.

    Parameters:
    - df: DataFrame containing the census data.
    - valid_years: List or range of years to fetch data for (e.g., range(2012,2023) for 2012 to 2022)
    - categories: List of category prefixes (e.g., ['HighSchoolEdu_Population', 'DoctorateEdu_Population']).
    - total_prefix: Prefix for the total column without year (e.g., 'Total_Population').

    Returns:
    - percentage_df: DataFrame with the specified category columns replaced by percentage values instead of raw counts.
    """
    percentage_df = df.copy()
    for year in valid_years:
        total_col = f'{total_prefix}_{year}'
        for category in categories:
            col = f'{category}_{year}'
            percentage_df[col] = (df[col] / df[total_col])
    return percentage_df


In [ ]:
def get_row(tract):
    """
    Parameters:
    - tract: The census tract for which to retrieve the row.

    Returns:
    - Row of data for the specified census tract.
    """

    return final_data[final_data['Census_Tract'] == tract].iloc[0]

# Median household income and median house value

In [ ]:
# Initialize DataFrames for each variable to store yearly data
income_data_all = fetch_census_data_by_field(
    state_fips,
    county_fips,
    valid_years,
    field='B19013_001E',
    field_name='Median_Household_Income'
)  # Fields and subfields obtained from ACS5 Census data

house_value_data_all = fetch_census_data_by_field(
    state_fips,
    county_fips,
    valid_years,
    field='B25077_001E',
    field_name='Median_House_Value'
)


# Merge income and house value data into a single table
final_data = pd.merge(income_data_all, house_value_data_all, on=['State_FIPS', 'County_FIPS', 'Census_Tract'], how='outer')

# Generate Unique ID directly in the table
final_data['Unique_ID'] = final_data['State_FIPS'] + final_data['County_FIPS'] + final_data['Census_Tract']

# Reorder columns to place Unique_ID first
cols = ['Unique_ID'] + [col for col in final_data.columns if col != 'Unique_ID']
final_data = final_data[cols]

# Define desired column order
desired_column_order = ['Unique_ID', 'State_FIPS', 'County_FIPS', 'Census_Tract']

# Dynamically add the income and house value columns
for year in valid_years:
    desired_column_order.extend([f'Median_Household_Income_{year}'])

for year in valid_years:
    desired_column_order.extend([f'Median_House_Value_{year}'])

# Apply the desired column order
final_data = final_data[desired_column_order]



In [ ]:
# Define the rows for each of the old census tracts
row_3302 = get_row('003302')
row_3705 = get_row('003705')
row_4201 = get_row('004201')
row_4301 = get_row('004301')
row_4304 = get_row('004304')
row_4310 = get_row('004310')
row_4405 = get_row('004405')
row_2400 = get_row('002400')

# Define the dictionary mapping new census tracts to their corresponding old row
tract_values = {
    ('003303', '003304'): row_3302,
    ('003707', '003708'): row_3705,
    ('004205', '004206'): row_4201,
    ('004313', '004314'): row_4301,
    ('004315', '004316'): row_4304,
    ('004311', '004312'): row_4310,
    ('004408', '004409'): row_4405,
    ('980000',): row_2400 # this row only changed in number, did not split
}

# Call the function to update the census data
columns_to_update = [f'Median_Household_Income_{year}' for year in valid_years] + [f'Median_House_Value_{year}' for year in valid_years] # Add other median variables here
final_data = update_median_census_tracts(final_data, tract_values, columns_to_update, years=range(2010, 2020))

# Additional row filtering - these rows contained only NA values or -666666666 (meaning that the data was hidden for privacy reasons)
final_data = final_data[final_data['Census_Tract'] != '002400']
final_data = final_data[final_data['Census_Tract'] != '980000']

# Save the complete data to the specified path
file_path = dir + 'median_household_income_and_median_house_value.csv'
final_data.to_csv(file_path, index=False)

# Print success message with file path
print(f"Data successfully saved to {file_path}")

Data successfully saved to /content/drive/MyDrive/HousingWorkplace/NEW and IMPROVED!! Mercer County Housing Analysis/Assets/median_household_income_and_median_house_value.csv


In [ ]:
print(final_data['Census_Tract'].unique())  # Print unique values in the 'Census_Tract' column
print('3302' in final_data['Census_Tract'].unique())  # Check if '3302' (one of the tracts removed) is present

['000100' '000200' '000300' '000400' '000500' '000600' '000700' '000800'
 '000900' '001000' '001101' '001102' '001200' '001300' '001401' '001402'
 '001500' '001600' '001700' '001800' '001900' '002000' '002100' '002200'
 '002500' '002601' '002602' '002701' '002702' '002800' '002902' '002903'
 '002904' '003001' '003002' '003003' '003004' '003006' '003007' '003008'
 '003009' '003100' '003201' '003202' '003301' '003303' '003304' '003400'
 '003500' '003601' '003602' '003703' '003704' '003706' '003707' '003708'
 '003800' '003902' '003903' '003904' '003905' '004000' '004203' '004204'
 '004205' '004206' '004306' '004307' '004309' '004311' '004312' '004313'
 '004314' '004315' '004316' '004403' '004404' '004406' '004407' '004408'
 '004409' '004501' '004502']
False


# Race Populations

In [ ]:
# Define fields for different population categories
fields = {
    'Total_Population': 'B02001_001E',
    'White_Population': 'B02001_002E',
    'Black_Population': 'B02001_003E',
    'Native_Population': 'B02001_004E',
    'Asian_Population': 'B02001_005E',
    'Hawaiian_Population': 'B02001_006E',
    'Other_Population': 'B02001_007E',
    'Two_Population': 'B02001_008E'
}

# Fetch data for all fields
population_dataframes = {}
for field_name, field in fields.items():
    population_dataframes[field_name] = fetch_census_data_by_field(
        state_fips,
        county_fips,
        valid_years,
        field,
        field_name
    )


# Merge all population data into a single DataFrame
final_data = None
for df in population_dataframes.values():
    if final_data is None:
        final_data = df
    else:
        final_data = pd.merge(final_data, df, on=['State_FIPS', 'County_FIPS', 'Census_Tract'], how='outer')

# Generate Unique ID directly in the table
final_data['Unique_ID'] = final_data['State_FIPS'] + final_data['County_FIPS'] + final_data['Census_Tract']

# Reorder columns to place Unique_ID first
cols = ['Unique_ID'] + [col for col in final_data.columns if col != 'Unique_ID']
final_data = final_data[cols]

# Define desired column order
desired_column_order = ['Unique_ID', 'State_FIPS', 'County_FIPS', 'Census_Tract']
for field_name in fields.keys():
    for year in valid_years:
        desired_column_order.append(f'{field_name}_{year}')

# Apply the desired column order
final_data = final_data[desired_column_order]

In [ ]:
# Define the rows for each of the old census tracts
row_3302 = get_row('003302')
row_3705 = get_row('003705')
row_4201 = get_row('004201')
row_4301 = get_row('004301')
row_4304 = get_row('004304')
row_4310 = get_row('004310')
row_4405 = get_row('004405')
row_2400 = get_row('002400')

# Define the dictionary mapping new census tracts to their corresponding old row
tract_values = {
    ('003303', '003304'): row_3302,
    ('003707', '003708'): row_3705,
    ('004205', '004206'): row_4201,
    ('004313', '004314'): row_4301,
    ('004315', '004316'): row_4304,
    ('004311', '004312'): row_4310,
    ('004408', '004409'): row_4405,
    ('980000',): row_2400 # this row only changed in number, did not split
}

# Define the columns to be updated
population_types = ['Total', 'White', 'Black', 'Native', 'Asian', 'Hawaiian', 'Other', 'Two']
columns_to_update = [f'{pop}_Population_{year}' for pop in population_types for year in valid_years]

# Call the function to update the census data
final_data = update_sum_census_tracts(final_data, tract_values, columns_to_update, years=range(2010, 2020))

# Save the complete data to the specified path
file_path = dir + 'race_populations.csv'
final_data.to_csv(file_path, index=False)

# Print success message with file path
print(f"Data successfully saved to {file_path}")

Data successfully saved to /content/drive/MyDrive/HousingWorkplace/NEW and IMPROVED!! Mercer County Housing Analysis/Assets/race_populations.csv


In [ ]:
race_data = pd.read_csv(dir + 'race_populations.csv')
percentage_data = race_data.copy()  # Start with a copy of the original data

for year in valid_years:
    # Calculate percentages for each race category
    percentage_data[f'White_Population_{year}'] = (race_data[f'White_Population_{year}'] / race_data[f'Total_Population_{year}'])
    percentage_data[f'Black_Population_{year}'] = (race_data[f'Black_Population_{year}'] / race_data[f'Total_Population_{year}'])
    percentage_data[f'Native_Population_{year}'] = (race_data[f'Native_Population_{year}'] / race_data[f'Total_Population_{year}'])
    percentage_data[f'Asian_Population_{year}'] = (race_data[f'Asian_Population_{year}'] / race_data[f'Total_Population_{year}'])
    percentage_data[f'Hawaiian_Population_{year}'] = (race_data[f'Hawaiian_Population_{year}'] / race_data[f'Total_Population_{year}'])
    percentage_data[f'Other_Population_{year}'] = (race_data[f'Other_Population_{year}'] / race_data[f'Total_Population_{year}'])
    percentage_data[f'Two_Population_{year}'] = (race_data[f'Two_Population_{year}'] / race_data[f'Total_Population_{year}'])


file_path = dir + 'race_populations_percentages.csv'
percentage_data.to_csv(file_path, index=False)
print(f"Percentage data successfully saved to {file_path}")

Percentage data successfully saved to /content/drive/MyDrive/HousingWorkplace/NEW and IMPROVED!! Mercer County Housing Analysis/Assets/race_populations_percentages.csv


# Educational Attainment

In [ ]:
# Define fields for different population categories
fields = {
    'Total_Population': 'B15003_001E',
    'NoEdu_Population': 'B15003_002E',
    'NurseryEdu_Population': 'B15003_003E',
    'KindergartenEdu_Population': 'B15003_004E',
    '1stEdu_Population': 'B15003_005E',
    '2ndEdu_Population': 'B15003_006E',
    '3rdEdu_Population': 'B15003_007E',
    '4thEdu_Population': 'B15003_008E',
    '5thEdu_Population': 'B15003_009E',
    '6thEdu_Population': 'B15003_010E',
    '7thEdu_Population': 'B15003_011E',
    '8thEdu_Population': 'B15003_012E',
    '9thEdu_Population': 'B15003_013E',
    '10thEdu_Population': 'B15003_014E',
    '11thEdu_Population': 'B15003_015E',
    '12thEdu_Population': 'B15003_016E',
    'HighSchoolEdu_Population': 'B15003_017E',
    'GEDEdu_Population': 'B15003_018E',
    'College1Edu_Population': 'B15003_019E',
    'College2Edu_Population': 'B15003_020E',
    'AssociateEdu_Population': 'B15003_021E',
    'BachelorEdu_Population': 'B15003_022E',
    'MasterEdu_Population': 'B15003_023E',
    'ProfessionalEdu_Population': 'B15003_024E',
    'DoctorateEdu_Population': 'B15003_025E',
}


# Fetch data for all fields
edu_dataframes = {}
for field_name, field in fields.items():
    print(f"Fetching data for {field_name}...")
    edu_dataframes[field_name] = fetch_census_data_by_field(
        state_fips,
        county_fips,
        valid_years,
        field,
        field_name
    )


# Merge all population data into a single DataFrame
final_data = None
for df in edu_dataframes.values():
    if final_data is None:
        final_data = df
    else:
        final_data = pd.merge(final_data, df, on=['State_FIPS', 'County_FIPS', 'Census_Tract'], how='outer')

# Generate Unique ID directly in the table
final_data['Unique_ID'] = final_data['State_FIPS'] + final_data['County_FIPS'] + final_data['Census_Tract']

# Reorder columns to place Unique_ID first
cols = ['Unique_ID'] + [col for col in final_data.columns if col != 'Unique_ID']
final_data = final_data[cols]

# Define desired column order
desired_column_order = ['Unique_ID', 'State_FIPS', 'County_FIPS', 'Census_Tract']
for field_name in fields.keys():
    for year in valid_years:
        desired_column_order.append(f'{field_name}_{year}')

# Apply the desired column order
final_data = final_data[desired_column_order]

Fetching data for Total_Population...
Fetching data for NoEdu_Population...
Fetching data for NurseryEdu_Population...
Fetching data for KindergartenEdu_Population...
Fetching data for 1stEdu_Population...
Fetching data for 2ndEdu_Population...
Fetching data for 3rdEdu_Population...
Fetching data for 4thEdu_Population...
Fetching data for 5thEdu_Population...
Fetching data for 6thEdu_Population...
Fetching data for 7thEdu_Population...
Fetching data for 8thEdu_Population...
Fetching data for 9thEdu_Population...
Fetching data for 10thEdu_Population...
Fetching data for 11thEdu_Population...
Fetching data for 12thEdu_Population...
Fetching data for HighSchoolEdu_Population...
Fetching data for GEDEdu_Population...
Fetching data for College1Edu_Population...
Fetching data for College2Edu_Population...
Fetching data for AssociateEdu_Population...
Fetching data for BachelorEdu_Population...
Fetching data for MasterEdu_Population...
Fetching data for ProfessionalEdu_Population...
Fetching d

In [ ]:
# Define the rows for each of the old census tracts
row_3302 = get_row('003302')
row_3705 = get_row('003705')
row_4201 = get_row('004201')
row_4301 = get_row('004301')
row_4304 = get_row('004304')
row_4310 = get_row('004310')
row_4405 = get_row('004405')
row_2400 = get_row('002400')

# Define the dictionary mapping new census tracts to their corresponding old row
tract_values = {
    ('003303', '003304'): row_3302,
    ('003707', '003708'): row_3705,
    ('004205', '004206'): row_4201,
    ('004313', '004314'): row_4301,
    ('004315', '004316'): row_4304,
    ('004311', '004312'): row_4310,
    ('004408', '004409'): row_4405,
    ('980000',): row_2400 # this row only changed in number, did not split
}

# Define the columns to be updated
edu_types = ['Total', 'NoEdu', 'NurseryEdu', 'KindergartenEdu', '1stEdu', '2ndEdu', '3rdEdu', '4thEdu', '5thEdu', '6thEdu', '7thEdu', '8thEdu', '9thEdu', '10thEdu', '11thEdu', '12thEdu', 'HighSchoolEdu', 'GEDEdu', 'College1Edu', 'College2Edu', 'AssociateEdu', 'BachelorEdu', 'MasterEdu', 'ProfessionalEdu', 'DoctorateEdu']
columns_to_update = [f'{edu}_Population_{year}' for edu in edu_types for year in valid_years]

# Call the function to update the census data
final_data = update_sum_census_tracts(final_data, tract_values, columns_to_update, years=range(2010, 2020))

# Save the complete data to the specified path
file_path = dir + 'educational_attainment.csv'
final_data.to_csv(file_path, index=False)

# Print success message with file path
print(f"Data successfully saved to {file_path}")

Data successfully saved to /content/drive/MyDrive/HousingWorkplace/NEW and IMPROVED!! Mercer County Housing Analysis/Assets/educational_attainment.csv


In [ ]:
edu_data = pd.read_csv(dir + 'educational_attainment.csv')
percentage_data = edu_data.copy()  # Start with a copy of the original data

# Variable list
base_vars = [
    'Total', 'NoEdu', 'NurseryEdu', 'KindergartenEdu', '1stEdu', '2ndEdu', '3rdEdu',
    '4thEdu', '5thEdu', '6thEdu', '7thEdu', '8thEdu', '9thEdu', '10thEdu', '11thEdu',
    '12thEdu', 'HighSchoolEdu', 'GEDEdu', 'College1Edu', 'College2Edu',
    'AssociateEdu', 'BachelorEdu', 'MasterEdu', 'ProfessionalEdu', 'DoctorateEdu'
]

# Append _Population to each
categories = [var + '_Population' for var in base_vars if var != 'Total']

# Run function — using 'Total_Population' as total_prefix
percentage_data = calculate_percentages(edu_data, valid_years, categories, 'Total_Population')


file_path = dir + 'educational_attainment_percentages.csv'
percentage_data.to_csv(file_path, index=False)
print(f"Percentage data successfully saved to {file_path}")

Percentage data successfully saved to /content/drive/MyDrive/HousingWorkplace/NEW and IMPROVED!! Mercer County Housing Analysis/Assets/educational_attainment_percentages.csv


# Other Variables

These are other variables that could be taken into consideration for their impacts on house value. Note that not all variables here are recorded in the U.S. Census Bureau's American Community Survey, and other methods of cleaning the data will be necessary for other types of statistics, like means.
- Average household size
-School quality
-Age of residents
-Income
-Family size
-Race
-Crime rates
-Policies
-Distance from economic centers or malls
-Distance from highways and train stations, public transport etc.
-Interest rates
-Unemployment rates
-Inflation rates
-Air quality, noise pollution
-New intrastructure construction
-Languages spoken
-Migration patterns
-New businesses/innovation
-Number of gas stations
-Number of fast food restaurants